# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd374bd32e0>
├── 'a' --> tensor([[ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605]])
└── 'x' --> <FastTreeValue 0x7fd374bd3250>
    └── 'c' --> tensor([[ 0.1660, -0.5548, -0.1921, -0.2606],
                        [-0.8941,  0.6323,  0.0318,  1.5649],
                        [-0.9146, -0.4820, -1.2256, -0.2821]])

In [4]:
t.a

tensor([[ 0.9012,  1.3270,  0.4984],
        [-1.3039,  0.0713,  1.4605]])

In [5]:
%timeit t.a

61.3 ns ± 0.0201 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd374bd32e0>
├── 'a' --> tensor([[1.2694, 0.5320, 0.3360],
│                   [0.2763, 0.3718, 1.0325]])
└── 'x' --> <FastTreeValue 0x7fd374bd3250>
    └── 'c' --> tensor([[ 0.1660, -0.5548, -0.1921, -0.2606],
                        [-0.8941,  0.6323,  0.0318,  1.5649],
                        [-0.9146, -0.4820, -1.2256, -0.2821]])

In [7]:
%timeit t.a = new_value

58.8 ns ± 0.0364 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9012,  1.3270,  0.4984],
               [-1.3039,  0.0713,  1.4605]]),
    x: Batch(
           c: tensor([[ 0.1660, -0.5548, -0.1921, -0.2606],
                      [-0.8941,  0.6323,  0.0318,  1.5649],
                      [-0.9146, -0.4820, -1.2256, -0.2821]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9012,  1.3270,  0.4984],
        [-1.3039,  0.0713,  1.4605]])

In [11]:
%timeit b.a

54.2 ns ± 0.0352 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8442, -0.4606, -1.0661],
               [-0.6941, -1.8023, -0.5685]]),
    x: Batch(
           c: tensor([[ 0.1660, -0.5548, -0.1921, -0.2606],
                      [-0.8941,  0.6323,  0.0318,  1.5649],
                      [-0.9146, -0.4820, -1.2256, -0.2821]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.214 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

836 ns ± 0.329 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 60.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 742 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 338 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd2bf58c5b0>
├── 'a' --> tensor([[[ 0.9012,  1.3270,  0.4984],
│                    [-1.3039,  0.0713,  1.4605]],
│           
│                   [[ 0.9012,  1.3270,  0.4984],
│                    [-1.3039,  0.0713,  1.4605]],
│           
│                   [[ 0.9012,  1.3270,  0.4984],
│                    [-1.3039,  0.0713,  1.4605]],
│           
│                   [[ 0.9012,  1.3270,  0.4984],
│                    [-1.3039,  0.0713,  1.4605]],
│           
│                   [[ 0.9012,  1.3270,  0.4984],
│                    [-1.3039,  0.0713,  1.4605]],
│           
│                   [[ 0.9012,  1.3270,  0.4984],
│                    [-1.3039,  0.0713,  1.4605]],
│           
│                   [[ 0.9012,  1.3270,  0.4984],
│                    [-1.3039,  0.0713,  1.4605]],
│           
│                   [[ 0.9012,  1.3270,  0.4984],
│                    [-1.3039,  0.0713,  1.4605]]])
└── 'x' --> <FastTreeValue 0x7fd2bee82070>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd2bee70e20>
├── 'a' --> tensor([[ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605],
│                   [ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605],
│                   [ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605],
│                   [ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605],
│                   [ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605],
│                   [ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605],
│                   [ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605],
│                   [ 0.9012,  1.3270,  0.4984],
│                   [-1.3039,  0.0713,  1.4605]])
└── 'x' --> <FastTreeValue 0x7fd2d0592fa0>
    └── 'c' --> tensor([[ 0.1660, -0.5548, -0.1921, -0.2606],
                        [-0.8941,  0.6323,  0.0318,  1.5649],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 75.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.1660, -0.5548, -0.1921, -0.2606],
                       [-0.8941,  0.6323,  0.0318,  1.5649],
                       [-0.9146, -0.4820, -1.2256, -0.2821]],
              
                      [[ 0.1660, -0.5548, -0.1921, -0.2606],
                       [-0.8941,  0.6323,  0.0318,  1.5649],
                       [-0.9146, -0.4820, -1.2256, -0.2821]],
              
                      [[ 0.1660, -0.5548, -0.1921, -0.2606],
                       [-0.8941,  0.6323,  0.0318,  1.5649],
                       [-0.9146, -0.4820, -1.2256, -0.2821]],
              
                      [[ 0.1660, -0.5548, -0.1921, -0.2606],
                       [-0.8941,  0.6323,  0.0318,  1.5649],
                       [-0.9146, -0.4820, -1.2256, -0.2821]],
              
                      [[ 0.1660, -0.5548, -0.1921, -0.2606],
                       [-0.8941,  0.6323,  0.0318,  1.5649],
                       [-0.9146, -0.4820, -1.2256, -0.2821]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 316 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.1660, -0.5548, -0.1921, -0.2606],
                      [-0.8941,  0.6323,  0.0318,  1.5649],
                      [-0.9146, -0.4820, -1.2256, -0.2821],
                      [ 0.1660, -0.5548, -0.1921, -0.2606],
                      [-0.8941,  0.6323,  0.0318,  1.5649],
                      [-0.9146, -0.4820, -1.2256, -0.2821],
                      [ 0.1660, -0.5548, -0.1921, -0.2606],
                      [-0.8941,  0.6323,  0.0318,  1.5649],
                      [-0.9146, -0.4820, -1.2256, -0.2821],
                      [ 0.1660, -0.5548, -0.1921, -0.2606],
                      [-0.8941,  0.6323,  0.0318,  1.5649],
                      [-0.9146, -0.4820, -1.2256, -0.2821],
                      [ 0.1660, -0.5548, -0.1921, -0.2606],
                      [-0.8941,  0.6323,  0.0318,  1.5649],
                      [-0.9146, -0.4820, -1.2256, -0.2821],
                      [ 0.1660, -0.5548, -0.1921, -0.2606],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 229 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

316 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
